In [2]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import random
import time
from IPython.display import clear_output

In [3]:
data = pd.read_csv('../dataset/raw/raw_paysim.csv')
data.columns= data.columns.str.lower()
data

,step,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isfraud,isflaggedfraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,0.0
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,0.0
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,0.0
1577099,155,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,0.0


In [4]:
data.dropna()

,step,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isfraud,isflaggedfraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1577095,155,TRANSFER,214977.94,C624724710,0.0,0.00,C1294409003,831919.08,1046897.02,0.0,0.0
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,0.0
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,0.0
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,0.0


In [5]:
data.rename(columns = {'isfraud':'EVENT_LABEL'}, inplace = True)

cols = data.columns.tolist()
cols = cols[0:9] + [cols[10]] + [cols[9]]
data = data[cols]

data = data.replace({'EVENT_LABEL': 0}, 'legit')
data = data.replace({'EVENT_LABEL': 1}, 'fraud')
data

,step,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isflaggedfraud,EVENT_LABEL
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,legit
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,legit
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,0.0,fraud
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,0.0,fraud
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,legit
...,...,...,...,...,...,...,...,...,...,...,...
1577096,155,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,legit
1577097,155,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,legit
1577098,155,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,legit
1577099,155,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,legit


In [6]:
def isdigit(num):
    if num < 10:
        return '0' + str(num)
    else:
        return str(num)

In [7]:
event_timestamp = []
for i, row in data.iterrows():
    step = row['step']
    month = isdigit(step // 30 + 3)
    day = isdigit(30 if step % 30 == 0 else step % 30)
    hours = isdigit(random.randint(0, 23))
    minutes = isdigit(random.randint(0, 59))
    seconds = isdigit(random.randint(0, 59))
    timestamp = '2022-'+month+'-'+day+'T'+hours+':'+minutes+':'+seconds +'Z'
    event_timestamp.append(timestamp)
data['EVENT_TIMESTAMP'] = event_timestamp
data = data.drop('step', axis=1)
data

,type,amount,nameorig,oldbalanceorg,newbalanceorig,namedest,oldbalancedest,newbalancedest,isflaggedfraud,EVENT_LABEL,EVENT_TIMESTAMP
0,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.00,0.00,0.0,legit,2022-03-01T04:54:02Z
1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.00,0.00,0.0,legit,2022-03-01T12:44:09Z
2,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.00,0.00,0.0,fraud,2022-03-01T15:26:49Z
3,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.00,0.00,0.0,fraud,2022-03-01T08:15:23Z
4,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.00,0.00,0.0,legit,2022-03-01T16:08:29Z
...,...,...,...,...,...,...,...,...,...,...,...
1577096,TRANSFER,43450.49,C730444513,0.0,0.00,C814720871,165948.65,1270011.14,0.0,legit,2022-08-05T14:06:43Z
1577097,TRANSFER,49185.39,C1893451872,0.0,0.00,C1661673055,1899236.38,1733474.42,0.0,legit,2022-08-05T05:36:20Z
1577098,TRANSFER,120655.98,C746221708,0.0,0.00,C1786396642,6672282.71,6792938.70,0.0,legit,2022-08-05T19:06:07Z
1577099,TRANSFER,755659.76,C1834005599,0.0,0.00,C1985986684,1509822.54,2265482.30,0.0,legit,2022-08-05T09:38:38Z


In [8]:
data.to_csv('../dataset/prepared/prepared_paysim_ofi.csv', index=False)

In [9]:
s3 = boto3.resource('s3')
BUCKET = 'paysim-fraud-detection-bucket'

s3.Bucket(BUCKET).upload_file('../dataset/prepared/prepared_paysim_ofi.csv', 'prepared_paysim_ofi.csv')

In [4]:
fraudDetector = boto3.client('frauddetector')

response = fraudDetector.get_detectors()
print(response)

{'detectors': [], 'ResponseMetadata': {'RequestId': '6f331cb3-f459-4e34-a9a4-3f1bce5bdf66', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 08 Sep 2022 15:12:36 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '16', 'connection': 'keep-alive', 'x-amzn-requestid': '6f331cb3-f459-4e34-a9a4-3f1bce5bdf66'}, 'RetryAttempts': 0}}


In [17]:
fraudDetector.create_variable(
    name = 'amount1',
    variableType = 'PRICE',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)


fraudDetector.create_variable(
    name = 'isflaggedfraud1',
    variableType = 'CATEGORICAL',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)   

fraudDetector.create_variable(
    name = 'namedest1',
    variableType = 'FREE_FORM_TEXT',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)

fraudDetector.create_variable(
    name = 'nameorig1',
    variableType = 'FREE_FORM_TEXT',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)


fraudDetector.create_variable(
    name = 'newbalancedest1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'newbalanceorig1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'oldbalancedest1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'oldbalanceorg1',
    variableType = 'NUMERIC',
    dataSource = 'EVENT',
    dataType = 'FLOAT',
    defaultValue = '0.0'
)

fraudDetector.create_variable(
    name = 'type1',
    variableType = 'PAYMENT_TYPE',
    dataSource = 'EVENT',
    dataType = 'STRING',
    defaultValue = '<unknown>'
)


{'ResponseMetadata': {'RequestId': '3c04ff12-725d-4a70-a00d-739789cdc035',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 Sep 2022 09:21:48 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3c04ff12-725d-4a70-a00d-739789cdc035'},
  'RetryAttempts': 0}}

In [41]:
fraudDetector.put_entity_type(
   name = 'paysimentity',
)

{'ResponseMetadata': {'RequestId': 'b920c46a-2747-4eaf-b236-b0b3785ad4c0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 Sep 2022 09:55:34 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b920c46a-2747-4eaf-b236-b0b3785ad4c0'},
  'RetryAttempts': 0}}

In [42]:
fraudDetector.put_label(
    name = 'fraud',
)

fraudDetector.put_label(
    name = 'legit',
)
 

{'ResponseMetadata': {'RequestId': 'bdba6450-698c-4a64-8f8e-b7b17b94cf14',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 Sep 2022 09:55:37 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bdba6450-698c-4a64-8f8e-b7b17b94cf14'},
  'RetryAttempts': 0}}

In [43]:
fraudDetector.put_event_type (
     name = 'paysim-event',
     eventVariables = ['amount', 'isflaggedfraud', 'namedest', 'nameorig','newbalancedest', 'newbalanceorig', 'oldbalancedest', 'oldbalanceorg', 'type'],
     labels = ['legit', 'fraud'],
     entityTypes = ['paysimentity'])            
        

{'ResponseMetadata': {'RequestId': '6ce07f91-258c-4c50-a87b-7c19ebf29e82',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 Sep 2022 09:55:42 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6ce07f91-258c-4c50-a87b-7c19ebf29e82'},
  'RetryAttempts': 0}}

In [10]:
fraudDetector.create_model (
       modelId = 'paysim_ofi_fraud_detection_model',
       eventTypeName = 'paysim-event',
       modelType = 'ONLINE_FRAUD_INSIGHTS')

{'ResponseMetadata': {'RequestId': '76298987-ab46-4fb3-a296-d3628c20697c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 08 Sep 2022 16:08:34 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '76298987-ab46-4fb3-a296-d3628c20697c'},
  'RetryAttempts': 0}}

In [12]:
fraudDetector.create_model_version (
         modelId = 'paysim_ofi_fraud_detection_model',
         modelType = 'ONLINE_FRAUD_INSIGHTS',
         trainingDataSource = 'EXTERNAL_EVENTS',
         trainingDataSchema = {
            'modelVariables' : ['amount', 'isflaggedfraud', 'namedest', 'nameorig','newbalancedest', 'newbalanceorig', 'oldbalancedest', 'oldbalanceorg', 'type'],
            'labelSchema' : {
               'labelMapper' : {
                   'FRAUD' : ['fraud'],
                   'LEGIT' : ['legit']
        }
    }
}, 
         externalEventsDetail = {
              'dataLocation' : 's3://paysim-fraud-detection-bucket/preprocessed_paysim.csv',
              'dataAccessRoleArn' : 'arn:aws:iam::235823010608:role/service-role/AmazonFraudDetector-DataAccessRole-1659514577130'
}
)

{'modelId': 'paysim_ofi_fraud_detection_model',
 'modelType': 'ONLINE_FRAUD_INSIGHTS',
 'modelVersionNumber': '1.0',
 'status': 'TRAINING_IN_PROGRESS',
 'ResponseMetadata': {'RequestId': '6c0f106b-af78-466b-a8cc-ead6588e6b31',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 08 Sep 2022 16:13:41 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '141',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6c0f106b-af78-466b-a8cc-ead6588e6b31'},
  'RetryAttempts': 0}}